<a href="https://colab.research.google.com/github/user-1221/home-pa-algo/blob/main/algo_1.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Home-PA Scheduling Prototype

This notebook hosts a simplified allocator that scores suggestions by combining their `need` and `importance`, filters by proximity to available gaps, searches for the least-travel permutation, and assigns time blocks while respecting splitting rules.



## Workflow Overview

1. Evaluate each suggestion with `score = need + importance` (values are clamped to `[0, 1]`).
2. Filter to suggestions close enough to at least one gap boundary (start or end).
3. Greedily keep the highest scoring suggestions that fit within the total free minutes.
4. Enumerate all feasible orderings (up to a configurable limit) and pick the one that minimises travel from the first gap's start location to the final gap's end location.
5. Attempt to embed the ordered suggestions into the gaps while respecting boundary travel constraints; drop the lowest-score suggestion and retry if a layout fails.



In [ ]:
from __future__ import annotations

import itertools
import math
from dataclasses import dataclass, field
from typing import List, Optional, Sequence, Tuple

Coordinate = Tuple[float, float]


def clamp01(value: float) -> float:
    """Clamp a floating-point value into the inclusive range [0, 1]."""
    if value < 0.0:
        return 0.0
    if value > 1.0:
        return 1.0
    return value


def euclidean_distance(a: Coordinate, b: Coordinate) -> float:
    """Return the Euclidean distance between two 2D coordinates."""
    return math.hypot(a[0] - b[0], a[1] - b[1])


@dataclass(frozen=True)
class Gap:
    """Represents an available time window (gap) in the day."""

    gap_id: str
    duration: float  # minutes
    start_location: Coordinate
    end_location: Coordinate

    def __post_init__(self) -> None:
        if self.duration <= 0:
            raise ValueError(f"Gap '{self.gap_id}' duration must be positive.")


@dataclass
class Suggestion:
    """A suggestion that may be slotted into the schedule."""

    suggestion_id: str
    need: float
    importance: float
    base_duration: float
    total_duration: float
    location: Coordinate
    splittable: bool = False

    score: float = field(init=False)

    def __post_init__(self) -> None:
        if self.base_duration <= 0:
            raise ValueError(
                f"Suggestion '{self.suggestion_id}' base_duration must be positive."
            )
        if self.total_duration <= 0:
            raise ValueError(
                f"Suggestion '{self.suggestion_id}' total_duration must be positive."
            )
        if self.total_duration < self.base_duration:
            raise ValueError(
                f"Suggestion '{self.suggestion_id}' total_duration ({self.total_duration}) "
                f"cannot be smaller than base_duration ({self.base_duration})."
            )
        object.__setattr__(self, "score", clamp01(self.need) + clamp01(self.importance))

        if self.splittable:
            blocks = self.total_duration / self.base_duration
            if not math.isclose(blocks, round(blocks), rel_tol=1e-9, abs_tol=1e-9):
                raise ValueError(
                    f"Splittable suggestion '{self.suggestion_id}' must have a "
                    "total_duration that is an integer multiple of base_duration."
                )

    @property
    def blocks_required(self) -> int:
        """Return the number of base blocks needed to satisfy the total duration."""
        if self.splittable:
            return int(round(self.total_duration / self.base_duration))
        return 1


@dataclass
class ScheduledBlock:
    """Represents a scheduled slice of a suggestion embedded inside a gap."""

    suggestion_id: str
    gap_id: str
    start_offset: float  # minutes from the gap start
    duration: float
    location: Coordinate


@dataclass
class ScheduleResult:
    """Container for the overall scheduling outcome."""

    ordered_suggestions: List[Suggestion]
    scheduled_blocks: List[ScheduledBlock]
    travel_cost: float
    dropped_suggestions: List[Suggestion]
    unused_gap_time: float
    permutations_evaluated: int

    def is_feasible(self) -> bool:
        """Return True when at least one suggestion was successfully scheduled."""
        return bool(self.scheduled_blocks)


def total_gap_minutes(gaps: Sequence[Gap]) -> float:
    """Total available minutes across all gaps."""
    return sum(gap.duration for gap in gaps)


def suggestion_is_near_gap(
    suggestion: Suggestion, gap: Gap, max_distance: float
) -> bool:
    """Return True when the suggestion lies close to a gap boundary."""
    distance_to_start = euclidean_distance(suggestion.location, gap.start_location)
    distance_to_end = euclidean_distance(suggestion.location, gap.end_location)
    return min(distance_to_start, distance_to_end) <= max_distance


def suggestion_is_close_to_any_gap(
    suggestion: Suggestion, gaps: Sequence[Gap], max_distance: float
) -> bool:
    """Check whether the suggestion is within `max_distance` of a gap boundary."""
    if not gaps:
        return False
    return any(
        suggestion_is_near_gap(suggestion, gap, max_distance) for gap in gaps
    )


def greedy_select_candidates(
    suggestions: Sequence[Suggestion],
    gaps: Sequence[Gap],
    max_distance: float,
    *,
    tolerance: float = 1e-6,
) -> List[Suggestion]:
    """
    Greedily select high-score suggestions that can fit within the combined gap time.

    Filtering rules:
    - Suggestions must lie within `max_distance` of at least one gap boundary.
    - Suggestions are considered in score-descending order.
    - A suggestion is included only if its total duration can fit into the
      remaining global capacity (sum of gap durations minus already selected).
    """
    capacity = total_gap_minutes(gaps)
    remaining_capacity = capacity

    nearby_suggestions = [
        s for s in suggestions if suggestion_is_close_to_any_gap(s, gaps, max_distance)
    ]
    sorted_candidates = sorted(
        nearby_suggestions, key=lambda s: s.score, reverse=True
    )

    selected: List[Suggestion] = []
    for suggestion in sorted_candidates:
        required = suggestion.total_duration
        if required <= remaining_capacity + tolerance:
            selected.append(suggestion)
            remaining_capacity -= required
    return selected


def enumerate_best_order(
    suggestions: Sequence[Suggestion],
    start_location: Optional[Coordinate],
    end_location: Optional[Coordinate],
    *,
    permutation_limit: int = 8,
) -> Tuple[List[Suggestion], float, int]:
    """
    Examine all feasible orderings (up to factorial explosion) and return the
    sequence with minimum travel cost.

    Returns
    -------
    order:
        The best permutation as a list of suggestions.
    travel_cost:
        Sum of Euclidean distances between consecutive points (including the
        optional start/end anchors).
    permutations_checked:
        Number of permutations evaluated in the search.
    """
    n = len(suggestions)
    if n == 0:
        return [], 0.0, 0

    if n > permutation_limit:
        raise ValueError(
            f"Permutation search limited to {permutation_limit} suggestions, "
            f"received {n}. Drop lower-score items before ordering."
        )

    best_order: Optional[Tuple[Suggestion, ...]] = None
    best_cost = math.inf
    permutations_checked = 0

    for permutation in itertools.permutations(suggestions):
        permutations_checked += 1
        travel_cost = 0.0
        previous = start_location

        for suggestion in permutation:
            if previous is not None:
                travel_cost += euclidean_distance(previous, suggestion.location)
            previous = suggestion.location

        if previous is not None and end_location is not None:
            travel_cost += euclidean_distance(previous, end_location)

        if travel_cost < best_cost:
            best_cost = travel_cost
            best_order = permutation

    assert best_order is not None
    return list(best_order), best_cost, permutations_checked


def assign_order_to_gaps(
    ordered_suggestions: Sequence[Suggestion],
    gaps: Sequence[Gap],
    max_distance: float,
    *,
    tolerance: float = 1e-6,
) -> Optional[List[ScheduledBlock]]:
    """
    Attempt to embed the ordered suggestions into the available gaps.

    Placement rules:
    - Non-splittable suggestions must be scheduled entirely within a single
      gap that is geographically close enough.
    - Splittable suggestions may span multiple gaps, but each fragment must be
      at least `base_duration`.
    - Movement inside a gap honours the start/end event locations; when the
      distance to either boundary exceeds `max_distance`, the layout is rejected.
    """
    if not ordered_suggestions:
        return []

    schedule: List[ScheduledBlock] = []
    remaining_per_suggestion = {
        s.suggestion_id: s.total_duration for s in ordered_suggestions
    }

    suggestion_index = 0
    gap_index = 0
    current_cursor: Optional[Coordinate] = None
    gap_has_blocks = False

    while suggestion_index < len(ordered_suggestions):
        if gap_index >= len(gaps):
            return None

        gap = gaps[gap_index]
        if current_cursor is None:
            current_cursor = gap.start_location
            gap_has_blocks = False

        used_in_gap = sum(
            block.duration for block in schedule if block.gap_id == gap.gap_id
        )
        remaining_gap = gap.duration - used_in_gap

        if remaining_gap <= tolerance:
            if gap_has_blocks:
                distance_to_end = euclidean_distance(current_cursor, gap.end_location)
                if distance_to_end > max_distance + tolerance:
                    return None
            gap_index += 1
            current_cursor = None
            gap_has_blocks = False
            continue

        suggestion = ordered_suggestions[suggestion_index]
        remaining_need = remaining_per_suggestion[suggestion.suggestion_id]

        distance_from_cursor = euclidean_distance(current_cursor, suggestion.location)
        if distance_from_cursor > max_distance:
            if gap_has_blocks:
                distance_to_end = euclidean_distance(current_cursor, gap.end_location)
                if distance_to_end > max_distance + tolerance:
                    return None
            gap_index += 1
            current_cursor = None
            gap_has_blocks = False
            continue

        if not suggestion.splittable:
            if remaining_need <= remaining_gap + tolerance:
                start_offset = used_in_gap
                schedule.append(
                    ScheduledBlock(
                        suggestion_id=suggestion.suggestion_id,
                        gap_id=gap.gap_id,
                        start_offset=start_offset,
                        duration=remaining_need,
                        location=suggestion.location,
                    )
                )
                remaining_per_suggestion[suggestion.suggestion_id] = 0.0
                suggestion_index += 1
                current_cursor = suggestion.location
                gap_has_blocks = True
            else:
                if gap_has_blocks:
                    distance_to_end = euclidean_distance(current_cursor, gap.end_location)
                    if distance_to_end > max_distance + tolerance:
                        return None
                gap_index += 1
                current_cursor = None
                gap_has_blocks = False
            continue

        blocks_remaining = remaining_need / suggestion.base_duration
        blocks_remaining = round(blocks_remaining)
        blocks_available = math.floor(
            (remaining_gap + tolerance) / suggestion.base_duration
        )

        if blocks_available <= 0:
            if gap_has_blocks:
                distance_to_end = euclidean_distance(current_cursor, gap.end_location)
                if distance_to_end > max_distance + tolerance:
                    return None
            gap_index += 1
            current_cursor = None
            gap_has_blocks = False
            continue

        blocks_to_schedule = min(blocks_remaining, blocks_available)
        duration_to_schedule = blocks_to_schedule * suggestion.base_duration
        start_offset = used_in_gap

        schedule.append(
            ScheduledBlock(
                suggestion_id=suggestion.suggestion_id,
                gap_id=gap.gap_id,
                start_offset=start_offset,
                duration=duration_to_schedule,
                location=suggestion.location,
            )
        )

        remaining_need -= duration_to_schedule
        remaining_per_suggestion[suggestion.suggestion_id] = remaining_need
        current_cursor = suggestion.location
        gap_has_blocks = True

        if remaining_need <= tolerance:
            suggestion_index += 1
        else:
            distance_to_end = euclidean_distance(current_cursor, gap.end_location)
            if distance_to_end > max_distance + tolerance:
                return None
            gap_index += 1
            current_cursor = None
            gap_has_blocks = False

    if gap_index < len(gaps):
        gap = gaps[gap_index]
        if gap_has_blocks:
            exit_location = current_cursor if current_cursor is not None else gap.start_location
            if euclidean_distance(exit_location, gap.end_location) > max_distance + tolerance:
                return None

    return schedule


def schedule_suggestions(
    suggestions: Sequence[Suggestion],
    gaps: Sequence[Gap],
    *,
    max_distance: float = 3.0,
    permutation_limit: int = 8,
    tolerance: float = 1e-6,
) -> ScheduleResult:
    """
    High-level orchestration for the simplified scheduler.

    Parameters
    ----------
    suggestions:
        Iterable of candidate suggestions to consider.
    gaps:
        Free time windows in chronological order.
    max_distance:
        Maximum Euclidean distance allowed between a suggestion and the gap it
        occupies.
    permutation_limit:
        Upper bound on the number of suggestions for which we evaluate all
        permutations. Larger sets should be pruned by dropping the lowest-score
        suggestions.
    """
    gap_list = list(gaps)
    suggestion_list = list(suggestions)

    total_capacity = total_gap_minutes(gap_list)
    if not gap_list or total_capacity <= tolerance:
        return ScheduleResult(
            ordered_suggestions=[],
            scheduled_blocks=[],
            travel_cost=0.0,
            dropped_suggestions=list(suggestion_list),
            unused_gap_time=0.0,
            permutations_evaluated=0,
        )

    selected = greedy_select_candidates(
        suggestion_list, gap_list, max_distance, tolerance=tolerance
    )
    dropped: List[Suggestion] = [
        suggestion for suggestion in suggestion_list if suggestion not in selected
    ]

    permutations_checked_total = 0
    while selected:
        selected.sort(key=lambda s: s.score, reverse=True)

        if len(selected) > permutation_limit:
            dropped_candidate = selected.pop()
            dropped.append(dropped_candidate)
            continue

        start_location = gap_list[0].start_location if gap_list else None
        end_location = gap_list[-1].end_location if gap_list else None

        try:
            order, travel_cost, permutations_checked = enumerate_best_order(
                selected,
                start_location=start_location,
                end_location=end_location,
                permutation_limit=permutation_limit,
            )
        except ValueError:
            dropped_candidate = selected.pop()
            dropped.append(dropped_candidate)
            continue

        permutations_checked_total += permutations_checked

        schedule = assign_order_to_gaps(
            order, gap_list, max_distance, tolerance=tolerance
        )
        if schedule is not None:
            used_time = sum(block.duration for block in schedule)
            unused_time = max(0.0, total_capacity - used_time)
            return ScheduleResult(
                ordered_suggestions=order,
                scheduled_blocks=schedule,
                travel_cost=travel_cost,
                dropped_suggestions=dropped,
                unused_gap_time=unused_time,
                permutations_evaluated=permutations_checked_total,
            )

        dropped_candidate = selected.pop()
        dropped.append(dropped_candidate)

    return ScheduleResult(
        ordered_suggestions=[],
        scheduled_blocks=[],
        travel_cost=0.0,
        dropped_suggestions=dropped,
        unused_gap_time=total_capacity,
        permutations_evaluated=permutations_checked_total,
    )


def format_schedule(result: ScheduleResult) -> str:
    """Render a human-friendly multi-line summary of the schedule."""
    if not result.scheduled_blocks:
        return "No feasible schedule could be generated."

    lines = [
        "Final schedule:",
        "----------------",
    ]

    for block in result.scheduled_blocks:
        lines.append(
            f"- Gap {block.gap_id}: {block.suggestion_id} "
            f"({block.duration:.0f} min starting at +{block.start_offset:.0f} min)"
        )

    lines.append("")
    lines.append(f"Travel cost: {result.travel_cost:.2f} grid units.")
    lines.append(
        f"Dropped suggestions: {', '.join(s.suggestion_id for s in result.dropped_suggestions) or 'None'}"
    )
    lines.append(f"Unused gap time: {result.unused_gap_time:.0f} minutes.")
    lines.append(
        f"Permutations evaluated: {result.permutations_evaluated}"
    )
    return "\n".join(lines)



In [ ]:
sample_suggestions = [
    Suggestion(
        suggestion_id="exercise",
        need=0.85,
        importance=0.9,
        base_duration=30,
        total_duration=60,
        location=(2.6, 3.0),
        splittable=True,
    ),
    Suggestion(
        suggestion_id="meal_prep",
        need=0.9,
        importance=0.8,
        base_duration=60,
        total_duration=60,
        location=(2.0, 2.3),
        splittable=False,
    ),
    Suggestion(
        suggestion_id="call_mom",
        need=0.65,
        importance=0.95,
        base_duration=30,
        total_duration=30,
        location=(3.6, 2.8),
        splittable=False,
    ),
    Suggestion(
        suggestion_id="deep_work",
        need=0.7,
        importance=0.8,
        base_duration=30,
        total_duration=60,
        location=(4.6, 4.2),
        splittable=True,
    ),
    Suggestion(
        suggestion_id="groceries",
        need=0.7,
        importance=0.55,
        base_duration=45,
        total_duration=45,
        location=(8.5, 7.6),
        splittable=False,
    ),
    Suggestion(
        suggestion_id="meditation",
        need=0.5,
        importance=0.7,
        base_duration=20,
        total_duration=20,
        location=(3.3, 3.0),
        splittable=False,
    ),
    Suggestion(
        suggestion_id="language_practice",
        need=0.45,
        importance=0.75,
        base_duration=25,
        total_duration=50,
        location=(5.8, 5.8),
        splittable=True,
    ),
    Suggestion(
        suggestion_id="cleaning",
        need=0.55,
        importance=0.6,
        base_duration=15,
        total_duration=30,
        location=(7.2, 6.9),
        splittable=True,
    ),
    Suggestion(
        suggestion_id="read_book",
        need=0.4,
        importance=0.7,
        base_duration=20,
        total_duration=40,
        location=(6.6, 6.5),
        splittable=True,
    ),
]

sample_gaps = [
    Gap(gap_id="gap_1", duration=180, start_location=(1.2, 1.8), end_location=(3.6, 3.1)),
    Gap(gap_id="gap_2", duration=70, start_location=(3.8, 3.2), end_location=(4.0, 3.5)),
    Gap(gap_id="gap_3", duration=90, start_location=(6.8, 6.4), end_location=(9.0, 8.5)),
]

result = schedule_suggestions(
    sample_suggestions,
    sample_gaps,
    max_distance=3.6,
    permutation_limit=8,
)
print(format_schedule(result))

print("\nOrdered evaluation scores:")
for suggestion in result.ordered_suggestions:
    print(f"- {suggestion.suggestion_id}: {suggestion.score:.2f}")

print("\nDropped suggestions with scores:")
for suggestion in result.dropped_suggestions:
    print(f"- {suggestion.suggestion_id}: {suggestion.score:.2f}")

